In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('news_dataset.csv')

In [3]:
df.label.value_counts()

economics        9308
tech             9293
business         9258
finance          9167
beauty           9130
entertainment    9028
food             8876
sports           8876
politics         8719
science          8582
travel           8107
world            6211
health           5557
environment      5121
Name: label, dtype: int64

In [4]:
df.head()

,title,excerpt,summary,label
0,Veteran Broadcaster Dave Allen Joins Par Mar S...,Longtime Tri-State broadcaster and media perso...,There were 611 press releases posted in the la...,economics
1,High-Performance Computing Center Stuttgart Ce...,"STUTTGART, Germany, Oct. 6, 2021 -- Founded in...","STUTTGART, Germany, Oct. 6, 2021 — Founded in ...",environment
2,Hopes new tech could finally find MH370,New technology could finally solve the mystery...,It's the world's biggest aviation mystery but ...,travel
3,Married At First Sight's Alexis Economou looks...,The MAFS UK star showed off her toned figure i...,"Published: 07:14 EDT, 9 October 2021 | Updated...",entertainment
4,SyncThink's Eye-Tracking VR Receives Second FD...,Eye-Sync uses eye-tracking to evaluate brain f...,Eye-Sync -- a virtual reality system from neur...,economics
